In [1]:
import pandas as pd
import numpy as np
import duckdb

## Loading GL AND PR data 

In [2]:
gl_file_excel = pd.read_csv('gl.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_14792\1320238739.py:1: DtypeWarning: Columns (6,7,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  gl_file_excel = pd.read_csv('gl.csv')


In [3]:
gl_file_excel.shape

(92046, 19)

In [6]:
pr_file_excel = pd.read_csv("pr.csv")

In [7]:
pr_file_excel.shape

(89605, 63)

In [8]:
gl_file_excel.head(1)

,MY GSTIN,Type,Supplier Name,Supplier GSTIN/UIN,Invoice No,Invoice Date,Original Invoice No,Original Invoice date,Total Invoice value,Place of Supply,Total Taxable Amount,Integrated Tax Amount,Central Tax Amount,State Tax Amount,Cess Amount,ITC Available - Cess,Remarks,HL Remarks,Final Remarks
0,29AADCH3844D1ZO,B2B,SOLUTIONS INFINI TECHNOLOGIES (I) Pvt Ltd,29AANCS0353C1ZF,NaN,NaN,NaN,NaN,2001.55,Karnataka,2001.55,13.98,173.15,173.15,NaN,NaN,Supplier GSTIN missing+Supplier Tax Invoice No...,Ignore No GST ( Journal Entry),NaN


In [9]:
# Convert to datetime
gl_file_excel['Invoice Date'] = pd.to_datetime(gl_file_excel['Invoice Date'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
gl_file_excel['Invoice Date'] = gl_file_excel['Invoice Date'].dt.strftime('%d-%m-%Y')

In [10]:
# Convert to datetime
pr_file_excel['Document Date (PR)'] = pd.to_datetime(pr_file_excel['Document Date (PR)'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
pr_file_excel['Document Date (PR)'] = pr_file_excel['Document Date (PR)'].dt.strftime('%d-%m-%Y')

In [11]:
pr_file_excel.head(1)

,ITC Claim month,Matching Key,Match Status,Match Status Description,Matching at Pan/ GSTIN,Sub-Buckets,Mismatched Fields,Mismatched Fields Count,Supplier Name,Supplier GSTIN (PR),...,Financial Year,Voucher Number,Out of Range (2B),Out of Range (PR),Claimable ITC - CGST,Claimable ITC - SGST,Claimable ITC - IGST,Claimable ITC - Cess,CT Remarks,Deferred ITC Month
0,2024 01,f5552d7f441c141c,Exact,NaN,My Gstin and Supplier Gstin Level,Actioned with No Changes,Taxable Value,1,TRINITY PANEL,29AAMFT6309F1ZV,...,2023-24,Jan'24,False,False,1737.9,1737.9,0.0,0,NaN,NaN


In [12]:
gl_file = gl_file_excel.copy()

In [13]:
pr_file = pr_file_excel.copy()

In [14]:
gl_file.shape

(92046, 19)

In [15]:
pr_file.shape

(89605, 63)

In [16]:
gl = duckdb.sql("""
              select * 
              from (
              select 
                "Supplier GSTIN/UIN",
                "MY GSTIN",
                regexp_replace("Invoice No", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '')   as "Invoice No",
                 "Invoice Date" as "Invoice Date",
                  string_agg(case when "Remarks" is null then '-' else "Remarks" end , '-')as "Remark",
                  string_agg(case when "HL Remarks" is null then '-' else "HL Remarks" end,'-') as "HL Remark",
                    sum(coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0)) as "IGST (GL)",
                    sum(coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0)) as "CGST (GL)",
                    sum(coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0)) as "SGST (GL)",
                    sum(
                  coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0) + 
                  coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0) +
                  coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0) 
              
              ) as "Total Tax(GL)",
               from gl_file
              group by 1,2,3,4 )
              """)

In [17]:
gl.shape

(88093, 10)

In [18]:
print(f'IGST amount is gl is:-> {gl.df()["IGST (GL)"].sum()}')
print(f'CGST amount is gl is:-> {gl.df()["CGST (GL)"].sum()}')
print(f'SGST amount is gl is:-> {gl.df()["SGST (GL)"].sum()}')
print(f'Total amount is gl is:-> {gl.df()["Total Tax(GL)"].sum()}')

IGST amount is gl is:-> 153907928.52999997
CGST amount is gl is:-> 117670418.69000003
SGST amount is gl is:-> 114199400.66999999
Total amount is gl is:-> 385777747.8899999


In [19]:
# Rank the rows in pr_file
pr_ranked_1 = duckdb.sql("""
    SELECT "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        REGEXP_REPLACE("Document Number (PR)", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '') AS "Document Number (PR)",
        "Document Date (PR)",
        COALESCE(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "IGST (PR)",
        COALESCE(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "CGST (PR)",
        COALESCE(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "SGST (PR)",
        CAST(REPLACE("Total Tax (PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM pr_file
""")

pr_ranked = duckdb.sql("""
    SELECT 
    "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        REGEXP_REPLACE("Document Number (PR)", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '') AS "Document Number (PR)",
        "Document Date (PR)",
        COALESCE(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "IGST (PR)",
        COALESCE(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "CGST (PR)",
        COALESCE(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)",
        row_number() OVER (PARTITION BY "Supplier GSTIN (PR)", "My GSTIN (PR)", "Document Number (PR)", "Document Date (PR)" 
           order by "Document Number (PR)") AS rn
    FROM pr_ranked_1
""")
# Separate the ranked data into rank 1 and duplicates
pr_filtered =duckdb.sql("""
  
          SELECT 
        
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        REGEXP_REPLACE("Document Number (PR)", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '') AS "Document Number (PR)",
        "Document Date (PR)",
        COALESCE(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "IGST (PR)",
        COALESCE(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "CGST (PR)",
        COALESCE(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
        
    FROM pr_ranked
    where rn= 1
""")

In [19]:
pr_filtered.shape

(88218, 8)

In [20]:
gl_file = gl.df()

In [21]:
pr_filtered = pr_filtered.df()

In [25]:
# Perform the left join with indicator
merged_1 = pd.merge(gl_file, pr_filtered,
                  how='left',
                  left_on=[
                      'Supplier GSTIN/UIN', 
                      'MY GSTIN', 
                      'Invoice No', 
                      'Invoice Date'
                  ],
                  right_on=[
                      'Supplier GSTIN (PR)', 
                      'My GSTIN (PR)', 
                      'Document Number (PR)', 
                      'Document Date (PR)'
                  ],
                  indicator=True)

In [26]:
merged_result = duckdb.sql("""

select * from merged_1

""")

In [27]:
merged_result

┌────────────────────┬─────────────────┬──────────────────┬───┬───────────┬───────────┬───────────────┬───────────┐
│ Supplier GSTIN/UIN │    MY GSTIN     │    Invoice No    │ … │ CGST (PR) │ SGST (PR) │ Total Tax(PR) │  _merge   │
│      varchar       │     varchar     │     varchar      │   │  double   │  double   │    double     │  _merge   │
├────────────────────┼─────────────────┼──────────────────┼───┼───────────┼───────────┼───────────────┼───────────┤
│ 07AAKCR2308F1ZK    │ 29AADCH3844D1ZO │ 855/2023-24      │ … │       0.0 │       0.0 │       1016.46 │ both      │
│ 29ABFFM0843B1ZK    │ 29AADCH3844D1ZO │ 1554             │ … │      54.0 │      54.0 │         108.0 │ both      │
│ 29AAICM1239J1Z8    │ 29AADCH3844D1ZO │ MPPL/24606/23-24 │ … │    3652.0 │    3652.0 │        7304.0 │ both      │
│ 29AAICM1239J1Z8    │ 29AADCH3844D1ZO │ MPCN0350/23-24   │ … │    -79.65 │    -79.65 │        -159.3 │ both      │
│ 29DIAPM0989P2ZD    │ 29AADCH3844D1ZO │ MA26481/2023-24  │ … │    1809.

In [29]:
merged_result.df().to_csv('merged_csv.csv')

In [20]:
pr_duplicates = duckdb.sql("""
    
    SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
            coalesce(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "IGST (PR)",
             coalesce(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)),0) as  "CGST (PR)",
             coalesce(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "SGST (PR)",
         CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
       FROM pr_ranked
    WHERE rn >= 2
""")


# Add duplicates with appropriate remarks
duplicates_df = duckdb.sql("""
    SELECT
       NULL AS "Supplier GSTIN/UIN",
       NULL AS "MY GSTIN",
       NULL AS "Invoice No", 
       NULL AS "Invoice Date", 
       NULL as "IGST (GL)",
       Null as "CGST (GL)",
       Null as "SGST (GL)",
       NULL AS "total Tax(GL)",
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)", 
        "Document Number (PR)",
        "Document Date (PR)", 
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        "Total Tax(PR)", 
        null as "Remark",
        null as "HL Remark",
        Null as "Matching keys",
       'Duplicate Entry In PR' AS "Remarks",
       Null as "Remarks_1"

    FROM pr_duplicates
""")


# Perform the full outer join and add remarks
result_1 = duckdb.sql("""
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered."CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark",
        gl_file."HL Remark",
        'SUPPLIER GSTIN-MY GSTIN-INVOICE NO-INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'All Match' as 'Remarks-1'
    FROM gl as gl_file
    FULL OUTER JOIN pr_filtered as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)"
""")

In [22]:
print(f'IGST amount is gl is:-> {result_1.df()["IGST (GL)"].sum()}')
print(f'CGST amount is gl is:-> {result_1.df()["CGST (GL)"].sum()}')
print(f'SGST amount is gl is:-> {result_1.df()["SGST (GL)"].sum()}')
print(f'Total Tax amount is gl is:-> {result_1.df()["Total Tax(GL)"].sum()}')

IGST amount is gl is:-> 153907928.53000003
CGST amount is gl is:-> 117670418.69
SGST amount is gl is:-> 114199400.67
Total Tax amount is gl is:-> 385777747.89000005


In [23]:
print(result_1.df()["IGST (GL)"].sum() - gl.df()["IGST (GL)"].sum())
print(result_1.df()["CGST (GL)"].sum() - gl.df()["CGST (GL)"].sum())
print(result_1.df()["SGST (GL)"].sum() - gl.df()["SGST (GL)"].sum())
print(result_1.df()["Total Tax(GL)"].sum() - gl.df()["Total Tax(GL)"].sum())

2.9802322387695312e-08
1.4901161193847656e-08
1.4901161193847656e-08
5.960464477539063e-08


In [24]:
result_1.shape

(105186, 21)

In [24]:
result_1.df().to_csv('result_1.csv', index = False)

In [25]:
matched_1 = duckdb.sql( """select * from result_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [26]:
matched_1.df().to_csv('matched_1.csv', index= False)

In [27]:
duplicates_df.df().to_csv('Duplicates.csv', index = False)

In [28]:
result_1 = pd.read_csv('result_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7736\2182478160.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_1 = pd.read_csv('result_1.csv')


In [30]:
print(f'IGST amount is gl is:-> {result_1["IGST (GL)"].sum()}')
print(f'CGST amount is gl is:-> {result_1["CGST (GL)"].sum()}')
print(f'SGST amount is gl is:-> {result_1["SGST (GL)"].sum()}')
print(f'Total amount is gl is:-> {result_1["Total Tax(GL)"].sum()}')

IGST amount is gl is:-> 153907928.53
CGST amount is gl is:-> 117670418.69
SGST amount is gl is:-> 114199400.66999999
Total amount is gl is:-> 385777747.89


In [47]:
pr_2 = duckdb.sql("""
    SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        sum("IGST (PR)") AS "IGST (PR)",
        sum("CGST (PR)") AS "CGST (PR)",
        sum("SGST (PR)") AS "SGST (PR)",
        sum(CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2))) AS "Total Tax(PR)"
    FROM result_1
        WHERE "Supplier GSTIN/UIN" is null
    GROUP BY 1,2,3,4
""")


In [48]:
pr_2.shape

(17094, 8)

In [52]:
gl_2 = duckdb.sql("""
SELECT 
       "Supplier GSTIN/UIN",
        "MY GSTIN",
        "Invoice No", 
        "Invoice Date", 
       string_agg("Remark",'') as "Remark",
        string_agg("HL Remark",'') as "HL Remark",
        sum("IGST (GL)") AS "IGST (GL)",
        sum("CGST (GL)") AS "CGST (GL)",
        sum("SGST (GL)") AS "SGST (GL)",
        sum("total Tax(GL)") as "total Tax(GL)"
 from result_1
 where  "Supplier GSTIN (PR)" is null 
 group by 1,2,3,4 """)

In [53]:
gl_2.shape

(16968, 10)

In [67]:
result_2 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)", 
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)", 
        pr_filtered."SGST (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."Total Tax(PR)",
        gl_file."Remark" as "Remark" ,
        gl_file."HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not match' as "Remarks_1"
    FROM gl_2 as gl_file
    FULL OUTER JOIN pr_2 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
""")


In [68]:
result_2.shape

(33709, 21)

In [69]:
result_2.df().to_csv('result_2.csv', index = False)

In [70]:
matched_2 = duckdb.sql( """select * from result_2
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)


In [72]:
matched_2.shape

(356, 21)

In [73]:
matched_2.df().to_csv('matched_2.csv', index= False)

In [74]:
matched_2 = pd.read_csv('matched_2.csv')

In [79]:
total_tax = matched_2["total Tax(GL)"].sum()
total_tax

22326475.060000002

In [60]:
result_2 = pd.read_csv('result_2.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7736\3392761352.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_2 = pd.read_csv('result_2.csv')


In [80]:
print(f'IGST amount is gl is:-> {result_2["IGST (GL)"].sum()}')
print(f'CGST amount is gl is:-> {result_2["CGST (GL)"].sum()}')
print(f'SGST amount is gl is:-> {result_2["SGST (GL)"].sum()}')
print(f'Total amount is gl is:-> {result_2["Total Tax(GL)"].sum()}')

CatalogException: Catalog Error: Scalar Function with name igst does not exist!
Did you mean "list"?

In [39]:
pr_2_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
         "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2
    WHERE "Supplier GSTIN/UIN" is null""")

In [40]:
gl_2_1 = duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_2
 where  "Supplier GSTIN (PR)" is null """)

In [41]:
result_2_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark",
        gl_file."HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO(Fuzzy)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not Match' as "Remarks_1"
    FROM gl_2_1 as gl_file
    FULL OUTER JOIN pr_2_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
    REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
     """)

In [42]:
result_2_1.df().to_csv('result_2_1.csv', index = False)

In [43]:
matched_2_1 = duckdb.sql( """select * from result_2_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )



In [44]:
matched_2_1.df().to_csv('matched_2_1.csv', index= False)

In [45]:
result_2_1 = pd.read_csv('result_2_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\1123820342.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_2_1 = pd.read_csv('result_2_1.csv')


In [46]:
pr_2_1_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
         "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [47]:
gl_2_1_1 = duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_2_1
 where  "Supplier GSTIN (PR)" is null """)

In [48]:
result_2_1_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER GSTIN ,MY GSTIN and Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Invoice Number and Date Not Match' as "Remarks_1"
    FROM gl_2_1_1 as gl_file
    FULL OUTER JOIN pr_2_1_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
""")


In [49]:
result_2_1_1.df().to_csv('result_2_1_1.csv', index = False)

In [50]:
matched_2_1_1 = duckdb.sql( """select * from result_2_1_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [51]:
matched_2_1_1.df().to_csv('matched_2_1_1.csv', index= False)

In [52]:
result_2_1_1 = pd.read_csv('result_2_1_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\3980419978.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_2_1_1 = pd.read_csv('result_2_1_1.csv')


In [53]:
pr_3 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
         "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2_1_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [54]:
gl_3= duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" 
 from result_2_1_1
 where  "Supplier GSTIN (PR)" is null """)


In [55]:
result_3 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark" ,
        gl_file."HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN Not Match' as "Remarks_1"
    FROM gl_3 as gl_file
    FULL OUTER JOIN pr_3 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")


In [56]:
result_3.df().to_csv('result_3.csv', index = False)

In [57]:
matched_3 = duckdb.sql( """select * from result_3
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [58]:
# matched_3.shape

In [59]:
matched_3.df().to_csv('matched_3.csv', index= False)

In [60]:
result_3= pd.read_csv('result_3.csv')

In [61]:
pr_4= duckdb.sql(""" 
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)", 
        "CGST (PR)",  
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_3
    WHERE "Supplier GSTIN/UIN" is null  """)

In [62]:
 gl_4 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" ,
 from result_3
 where  "Supplier GSTIN (PR)" is null """)

In [64]:
result_4 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date Not match' as "Remarks_1"
    FROM gl_4 as gl_file
    FULL OUTER JOIN pr_4 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    
""")


In [65]:
result_4.df().to_csv('result_4.csv', index = False)

In [66]:
matched_4 = duckdb.sql( """select * from result_4
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [67]:
matched_4.df().to_csv('matched_4.csv', index= False)

In [68]:
result_4 = pd.read_csv('result_4.csv')

In [69]:
pr_4_1  =duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4
    WHERE "Supplier GSTIN/UIN" is null  """)

In [70]:
 gl_4_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark" ,
    "HL Remark",
 from result_4
 where  "Supplier GSTIN (PR)" is null """)

In [71]:
result_4_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_4_1 as gl_file
    FULL OUTER JOIN pr_4_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
      
""")


In [72]:
result_4_1.df().to_csv('result_4_1.csv', index= False)

In [73]:
matched_4_1 = duckdb.sql( """select * from result_4_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [74]:
matched_4_1.df().to_csv('matched_4_1.csv', index= False)

In [75]:
result_4_1 = pd.read_csv('result_4_1.csv')

In [76]:
pr_4_1_1  =duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4_1
    WHERE "Supplier GSTIN/UIN" is null  """)

In [77]:
 gl_4_1_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark" ,
    "HL Remark",
 from result_4_1
 where  "Supplier GSTIN (PR)" is null """)

In [78]:
result_4_1_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'MY GSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_4_1_1 as gl_file
    FULL OUTER JOIN pr_4_1_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        """)


In [79]:
result_4_1_1.df().to_csv('result_4_1_1.csv', index= False)

In [80]:
matched_4_1_1 = duckdb.sql( """select * from result_4_1_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [81]:
matched_4_1_1.df().to_csv('matched_4_1_1.csv', index= False)

In [82]:
result_4_1_1 = pd.read_csv('result_4_1_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\3233401830.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_4_1_1 = pd.read_csv('result_4_1_1.csv')


In [85]:
pr_5 = duckdb.sql("""
         SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4_1_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [86]:
 gl_5 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark",
 from result_4_1_1
 where  "Supplier GSTIN (PR)" is null """)

In [87]:
result_5 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
       
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN Not Match' as "Remarks_1"
    FROM gl_5 as gl_file
    FULL OUTER JOIN pr_5 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")

In [88]:
result_5.df().to_csv('result_5.csv', index= False)

In [89]:
matched_5 = duckdb.sql( """select * from result_5
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [90]:
matched_5.df().to_csv('matched_5.csv', index= False)

In [91]:
result_5= pd.read_csv('result_5.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\3965734633.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_5= pd.read_csv('result_5.csv')


In [92]:
 pr_6 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_5
    WHERE "Supplier GSTIN/UIN" is null """)

In [93]:
gl_6 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
      "IGST (GL)",
      "CGST (GL)",
      "SGST (GL)",
      "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_5
 where  "Supplier GSTIN (PR)" is null """)

In [94]:
result_6 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and Date not match' as "Remarks_1"
    FROM gl_6 as gl_file
    FULL OUTER JOIN pr_6 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    
""")


In [95]:
result_6.df().to_csv('result_6.csv', index= False)

In [96]:
matched_6 = duckdb.sql( """select * from result_6
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [97]:
matched_6.df().to_csv('matched_6.csv', index= False)

In [98]:
result_6 = pd.read_csv('result_6.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\4250211578.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_6 = pd.read_csv('result_6.csv')


In [99]:
 pr_6_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6
    WHERE "Supplier GSTIN/UIN" is null """)

In [100]:
 gl_6_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6
 where  "Supplier GSTIN (PR)" is null """)

In [101]:
result_6_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
         pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Suppliers GSTIN and Date not match' as "Remarks_1"
    FROM gl_6_1 as gl_file
    FULL OUTER JOIN pr_6_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [102]:
result_6_1.df().to_csv('result_6_1.csv',index= False)

In [103]:
matched_6_1 = duckdb.sql( """select * from result_6_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [104]:
matched_6_1.df().to_csv('matched_6_1.csv', index= False)

In [105]:
result_6_1 = pd.read_csv('result_6_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\1076845252.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_6_1 = pd.read_csv('result_6_1.csv')


In [106]:
 pr_6_1_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6_1
    WHERE "Supplier GSTIN/UIN" is null """)

In [107]:
 gl_6_1_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6_1
 where  "Supplier GSTIN (PR)" is null """)

In [108]:
result_6_1_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'SupplierGSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_6_1_1 as gl_file
    FULL OUTER JOIN pr_6_1_1 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN"  = pr_filtered."My GSTIN (PR)"
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [109]:
result_6_1_1.df().to_csv('result_6_1_1.csv',index= False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [110]:
matched_6_1_1 = duckdb.sql( """select * from result_6_1_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [111]:
matched_6_1_1.df().to_csv('matched_6_1_1.csv', index= False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [112]:
result_6_1_1 = pd.read_csv('result_6_1_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\1442313742.py:1: DtypeWarning: Columns (0,1,2,3,8,9,10,11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_6_1_1 = pd.read_csv('result_6_1_1.csv')


In [113]:
 pr_7= duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6_1_1
    WHERE "Supplier GSTIN/UIN" is null  """)

In [111]:
# pr_7.shape

In [114]:
 gl_7 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6_1_1
 where  "Supplier GSTIN (PR)" is null """)

In [115]:
result_7 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and MY GSTIN not match' as "Remarks_1"
    FROM gl_7 as gl_file
    FULL OUTER JOIN pr_7 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")

In [116]:
result_7.df().to_csv('result_7.csv',index= False)

In [117]:
matched_7 = duckdb.sql( """select * from result_7
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [118]:
matched_7.df().to_csv('matched_7.csv', index= False)

In [119]:
result_7= pd.read_csv('result_7.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\4137335595.py:1: DtypeWarning: Columns (0,1,2,3,8,9,10,11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_7= pd.read_csv('result_7.csv')


In [120]:
 pr_8 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_7
    WHERE "Supplier GSTIN/UIN" is null  """)

In [121]:
gl_8 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
   "HL Remark" 
 from result_7
 where  "Supplier GSTIN (PR)" is null """)

In [122]:
result_8 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8 as gl_file
    FULL OUTER JOIN pr_8 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
""")


In [123]:
result_8.df().to_csv('result_8.csv',index= False)

In [124]:
matched_8 = duckdb.sql( """select * from result_8
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [125]:
matched_8.df().to_csv('matched_8.csv', index= False)

In [126]:
result_8 = pd.read_csv('result_8.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\1566926587.py:1: DtypeWarning: Columns (0,1,2,3,8,9,10,11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_8 = pd.read_csv('result_8.csv')


In [127]:
 pr_8_1  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8
    WHERE "Supplier GSTIN/UIN" is null """)

In [128]:
 gl_8_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_8
 where  "Supplier GSTIN (PR)" is null """)

In [129]:
result_8_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8_1 as gl_file
    FULL OUTER JOIN pr_8_1 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [130]:
result_8_1.df().to_csv('result_8_1.csv',index= False)

In [131]:
matched_8_1 = duckdb.sql( """select * from result_8_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [132]:
result_8_1 = pd.read_csv('result_8_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_7984\958001367.py:1: DtypeWarning: Columns (0,1,2,3,8,9,10,11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_8_1 = pd.read_csv('result_8_1.csv')


In [133]:
 pr_8_1_1  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8_1
    WHERE "Supplier GSTIN/UIN" is null 
    
    """)

In [134]:
gl_8_1_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
  from result_8_1
  where  "Supplier GSTIN (PR)" is null
 """ )

In [135]:
result_8_1_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_8_1_1 as gl_file
    FULL OUTER JOIN pr_8_1_1 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [136]:
result_8_1_1.df().to_csv('result_8_1_1.csv',index= False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [135]:
matched_8_1_1 = duckdb.sql( """select * from result_8_1_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [136]:
result_8_1_1 = pd.read_csv('result_8_1_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_6928\3049388475.py:1: DtypeWarning: Columns (0,1,2,3,8,9,10,11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_8_1_1 = pd.read_csv('result_8_1_1.csv')


In [137]:
pr_13  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8_1_1
    WHERE "Supplier GSTIN/UIN" is null """)

In [138]:
gl_13 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_8_1_1
 where  "Supplier GSTIN (PR)" is null """)

In [139]:
result_13 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
         'BUYER GSTIN/PAN, INOVICE NO, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN(PAN),MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_13 as gl_file
    FULL OUTER JOIN pr_13 as pr_filtered
    ON 
     (
            gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)" 
            OR SUBSTRING(gl_file."MY GSTIN", 3, 10) = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10)
        )
AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [140]:
result_13.df().to_csv('result_13.csv',index= False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


KeyboardInterrupt



In [ ]:
matched_13 = duckdb.sql( """select * from result_13
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
result_13 = pd.read_csv('result_13.csv')

In [ ]:
combined_sheet = duckdb.sql("""
                    select   * from matched_1
                    union all
                    select   * from matched_2
                    union all 
                    select   * from matched_2_1
                    union all 
                    select   * from matched_2_1_1
                    union all
                    select   * from matched_3 
                    union all 
                    select   * from matched_4
                    union all 
                    select   * from matched_4_1
                    union all 
                    select   * from matched_4_1_1
                    union all
                    select * from matched_5
                    union all
                    select * from matched_6
                    union all 
                    select * from matched_6_1
                    union all 
                    select * from matched_6_1_1
                    union all 
                    select * from matched_7
                    union all
                    select * from matched_8
                    union all 
                    select * from matched_8_1
                    union all 
                    select * from matched_8_1_1
                    union all
                    select * from result_13
                    
                     """)


In [ ]:
combined_sheet.df().to_csv('combined_sheet.csv', index = False)

In [ ]:
duplicates_df = pd.read_csv("Duplicates.csv")

In [ ]:
final_sheet = duckdb.sql("""
    select * from combined_sheet
    union all
    select * from duplicates_df
""")

In [ ]:
final_sheet.df().to_csv('final_sheet.csv', index= False)